In [1]:
import glob
import pandas as pd
import cv2
import gc
import numpy as np
import random
import imageio
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from urllib.request import urlopen
import wandb

BATCH_SIZE = 8
LEARNING_RATE = 0.0001
NUM_EPOCHS = 100
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_CLASSES = 101
CLIP_DURATION = 16
NUM_WORKERS = 2
PIN_MEMORY = True
PREFETCH_FACTOR = 2
EARLY_STOPPING_PATIENCE = 20

# Configuration class
wandb.login(key='<YOUR_KEY>')
wandb.init(
    project="the last",
    config={
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "epochs": NUM_EPOCHS,
        "clip_duration": CLIP_DURATION,
        "num_classes": NUM_CLASSES,
    }
)

class CFG:
    epochs = 200
    batch_size = 8
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    learning_rate = 0.00005
    num_workers = 2
    classes = [
        "ApplyEyeMakeup", "ApplyLipstick", "Archery", "BabyCrawling", "BalanceBeam",
        "BandMarching", "BaseballPitch", "Basketball", "BasketballDunk", "BenchPress",
        "Biking", "Billiards", "BlowDryHair", "BlowingCandles", "BodyWeightSquats",
        "Bowling", "BoxingPunchingBag", "BoxingSpeedBag", "BreastStroke", "BrushingTeeth",
        "CleanAndJerk", "CliffDiving", "CricketBowling", "CricketShot", "CuttingInKitchen",
        "Diving", "Drumming", "Fencing", "FieldHockeyPenalty", "FloorGymnastics",
        "FrisbeeCatch", "FrontCrawl", "GolfSwing", "Haircut", "HammerThrow",
        "Hammering", "HandstandPushups", "HandstandWalking", "HeadMassage", "HighJump",
        "HorseRace", "HorseRiding", "HulaHoop", "IceDancing", "JavelinThrow",
        "JugglingBalls", "JumpingJack", "JumpRope", "Kayaking", "Knitting",
        "LongJump", "Lunges", "MilitaryParade", "Mixing", "MoppingFloor",
        "Nunchucks", "ParallelBars", "PizzaTossing", "PlayingCello", "PlayingDaf",
        "PlayingDhol", "PlayingFlute", "PlayingGuitar", "PlayingPiano", "PlayingSitar",
        "PlayingTabla", "PlayingViolin", "PoleVault", "PommelHorse", "PullUps",
        "Punch", "PushUps", "Rafting", "RockClimbingIndoor", "RopeClimbing",
        "Rowing", "SalsaSpin", "ShavingBeard", "Shotput", "SkateBoarding",
        "Skiing", "Skijet", "SkyDiving", "SoccerJuggling", "SoccerPenalty",
        "StillRings", "SumoWrestling", "Surfing", "Swing", "TableTennisShot",
        "TaiChi", "TennisSwing", "ThrowDiscus", "TrampolineJumping", "Typing",
        "UnevenBars", "VolleyballSpiking", "WalkingWithDog", "WallPushups", "WritingOnBoard",
        "YoYo"
    ]
    videos_per_class = 50

# Custom Dataset class
class VideoDataset(Dataset):
    def __init__(self, file_paths, targets, transform=None):
        self.file_paths = file_paths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        video_frames = frames_from_video_file(self.file_paths[idx], n_frames=10)
        label = self.targets[idx]
        
        video_frames = torch.FloatTensor(video_frames)  # Shape: (num_frames, channels, height, width)
        
        if self.transform:
            video_frames = self.transform(video_frames)
        
        return video_frames, torch.tensor(label, dtype=torch.long)

def format_frames(frame, output_size):
    """Format frames to tensor with specified size"""
    frame = cv2.resize(frame, output_size)
    frame = frame / 255.0
    return frame

def frames_from_video_file(video_path, n_frames, output_size=(224, 224), frame_step=15):
    """Extract frames from video file"""
    result = []
    src = cv2.VideoCapture(str(video_path))
    
    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)
    need_length = 1 + (n_frames - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    ret, frame = src.read()
    result.append(format_frames(frame, output_size))

    for _ in range(n_frames - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
        if ret:
            frame = format_frames(frame, output_size)
            result.append(frame)
        else:
            result.append(np.zeros_like(result[0]))

    src.release()
    result = np.array(result)
    result = np.transpose(result, (0, 3, 1, 2))  # (T, C, H, W) format
    return result

# Model definition
# class VideoClassifier(nn.Module):
#     def __init__(self, num_classes):
#         super(VideoClassifier, self).__init__()
#         self.efficientnet = models.resnet50(pretrained=True)
#         self.efficientnet = nn.Sequential(*list(self.efficientnet.children())[:-1])
        
#         # Freeze the pre-trained parameters
#         for param in self.efficientnet.parameters():
#             param.requires_grad = False
            
#         self.temporal_pool = nn.AdaptiveAvgPool3d((1, 1, 1))
#         # Modified classifier structure
#         self.classifier = nn.Sequential(
#             nn.Linear(2048 * 10, 512),  # 1280 is EfficientNetB0's feature size, 10 is number of frames
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(512, num_classes)
#         )

#     def forward(self, x):
#         # x shape: (batch_size, num_frames, channels, height, width)
#         b, t, c, h, w = x.shape
        
#         # Process each frame through EfficientNet
#         frame_features = []
#         for i in range(t):
#             frame = x[:, i, :, :, :]  # (batch_size, channels, height, width)
#             features = self.efficientnet(frame)  # (batch_size, 1280, 1, 1)
#             frame_features.append(features.squeeze(-1).squeeze(-1))  # (batch_size, 1280)
        
#         # Concatenate features from all frames
#         x = torch.cat(frame_features, dim=1)  # (batch_size, 1280 * num_frames)
        
#         # Final classification
#         x = self.classifier(x)
#         return x

class VideoClassifier(nn.Module):
    def __init__(self, num_classes):
        super(VideoClassifier, self).__init__()
        
        # Sử dụng pre-trained ResNet50 để trích xuất đặc trưng từ mỗi frame
        self.resnet = models.resnet50(pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])  # Loại bỏ phần FC cuối cùng
        
        # Freeze the pre-trained parameters
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        # LSTM sẽ xử lý chuỗi các đặc trưng được trích xuất từ các frame
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.rnn = nn.LSTM(input_size=2048, hidden_size=512, num_layers=2, batch_first=True, bidirectional=True)
        
        # Lớp phân loại cuối cùng
        self.fc = nn.Linear(512 * 2, num_classes)  # LSTM bidirectional * 2 (for hidden and cell states)
        
        # Dropout để tránh overfitting
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # x shape: (batch_size, num_frames, channels, height, width)
        b, t, c, h, w = x.shape
        
        # Chuyển mỗi frame thành các đặc trưng (2048 dimensional)
        frame_features = []
        for i in range(t):
            frame = x[:, i, :, :, :]  # (batch_size, channels, height, width)
            features = self.resnet(frame)  # (batch_size, 2048, h', w')
            features = self.pool(features)
            features = features.flatten(1)  # Flatten thành (batch_size, 2048)
            frame_features.append(features)
        
        # Tạo một chuỗi các đặc trưng từ các frame
        x = torch.stack(frame_features, dim=1)  # (batch_size, num_frames, 2048)

        # LSTM xử lý chuỗi các đặc trưng frame
        x, _ = self.rnn(x)  # (batch_size, num_frames, 1024) --> vì LSTM là bidirectional
        
        # Lấy output cuối cùng của LSTM (hoặc có thể tính trung bình trên tất cả các frame)
        x = x[:, -1, :]  # Chỉ lấy output cuối cùng từ chuỗi
        
        # Dropout để tránh overfitting
        x = self.dropout(x)
        
        # Phân loại
        x = self.fc(x)  # Output (batch_size, num_classes)
        
        return x

# Training function
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for videos, labels in tqdm(dataloader):
        videos, labels = videos.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(videos)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

# Validation function
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for videos, labels in tqdm(dataloader):
            videos, labels = videos.to(device), labels.to(device)
            
            outputs = model(videos)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss = running_loss / len(dataloader)
    val_acc = 100. * correct / total
    return val_loss, val_acc, all_preds, all_labels

# Main training loop
def main():
    # Load dataset
    file_paths = []
    targets = []
    for i, cls in enumerate(CFG.classes):
        sub_file_paths = glob.glob(f"/kaggle/input/ucf101-action-recognition/train/{cls}/**.avi")[:CFG.videos_per_class]
        file_paths += sub_file_paths
        targets += [i] * len(sub_file_paths)

    # Split dataset
    train_paths, val_paths, train_targets, val_targets = train_test_split(
        file_paths, targets, test_size=0.2, random_state=42
    )

    # Create datasets and dataloaders
    train_dataset = VideoDataset(train_paths, train_targets)
    val_dataset = VideoDataset(val_paths, val_targets)

    train_loader = DataLoader(
        train_dataset, 
        batch_size=CFG.batch_size,
        shuffle=True,
        num_workers=CFG.num_workers
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=CFG.num_workers
    )

    # Initialize model
    model = VideoClassifier(num_classes=len(CFG.classes)).to(CFG.device)
    # model = VideoResNetClassifier(num_classes=len(CFG.classes)).to(CFG.device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=CFG.learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', patience=3, factor=0.5
    )

    # Training loop
    best_val_acc = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(CFG.epochs):
        print(f'Epoch {epoch+1}/{CFG.epochs}')
        
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, CFG.device
        )
        
        val_loss, val_acc, all_preds, all_labels = validate(
            model, val_loader, criterion, CFG.device
        )
        
        # Update learning rate
        scheduler.step(val_loss)
        
        wandb.log({
        "Epoch": epoch + 1,
        "Train Loss": train_loss,
        "Train Accuracy": train_acc,
        "Validation Loss": val_loss,
        "Validation Accuracy": val_acc,
        "Learning Rate": optimizer.param_groups[0]['lr']
        })
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            checkpoint_path = '/kaggle/working/model.pth'

            torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            # 'optimizer_state_dict': optimizer.state_dict(),
            # 'scheduler_state_dict': scheduler.state_dict(),
            # 'val_accuracy': val_acc
            }, checkpoint_path)

        # Update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f'Train Loss: {train_loss:.4f} Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} Val Acc: {val_acc:.2f}%')

        # Plot confusion matrix
        if epoch == CFG.epochs - 1:
            plot_confusion_matrix(all_labels, all_preds, CFG.classes)

    return model, history

def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(len(class_names), len(class_names)))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

if __name__ == "__main__":
    model, history = main()
    

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: thanh-20040803 (thanh-20040803-hanoi-university-of-science-and-technology). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241209_181135-2mcv82zy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sleek-mountain-4
wandb: ⭐️ View project at https://wandb.ai/thanh-20040803-hanoi-university-of-science-and-technology/the%20last
wandb: 🚀 View run at https://wandb.ai/t

Epoch 1/200


100%|██████████| 127/127 [00:33<00:00,  3.75it/s]


Train Loss: 4.4252 Train Acc: 4.93%
Val Loss: 4.0183 Val Acc: 11.49%
Epoch 2/200


100%|██████████| 127/127 [00:29<00:00,  4.38it/s]


...

Train Loss: 0.0954 Train Acc: 98.00%
Val Loss: 0.6030 Val Acc: 82.87%
Epoch 199/200


100%|██████████| 127/127 [00:29<00:00,  4.32it/s]


Train Loss: 0.0929 Train Acc: 98.04%
Val Loss: 0.6375 Val Acc: 80.99%
Epoch 200/200


100%|██████████| 127/127 [00:30<00:00,  4.23it/s]


Train Loss: 0.0915 Train Acc: 98.00%
Val Loss: 0.6090 Val Acc: 83.96%
